Jupyter Notebook that extracts the catchment area given a DEM (possbily stiched together in the other jupyter notebook) and the user input for drainage points. The script works in the local CRS of the DEM!!!!! Therefore, you have to adjust the snap tolerance according to your DEM (how accurate is it? Could there be a false basin?), your needs (how are sure are you about the pour point you set in the map). Most projected CRS are in meters, however if you work in a geographic CRS (eg WGS84), you have to adjust the snap tolerance accordingly (or even better work in a projected CRS). 

Adjust snap tolerance in the script

**Cell 1:** Install packages
Uncomment this cell and run if packages are missing

In [1]:
# !pip install whitebox folium geopandas rasterio shapely

**Cell 2:** Imports and workspace setup

In [30]:
import os
from whitebox.whitebox_tools import WhiteboxTools
import folium
from folium import plugins
import rasterio
import geopandas as gpd
from shapely.geometry import Point
import json
from rasterio.warp import transform_bounds
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS

**Cell 3:** Paths and helper function

In [32]:
# Base workspace path
workspace = r'C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis'
tif_dir = os.path.join(workspace, 'tif_edits')
os.makedirs(tif_dir, exist_ok=True)

shp_dir = os.path.join(workspace, 'shp_edits')
os.makedirs(shp_dir, exist_ok=True)

# Initialize WhiteboxTools
wbt = WhiteboxTools()
wbt.set_working_dir(workspace)

# Helper to make paths
def tif(name):
    return os.path.join(tif_dir, name + '.tif')

def shp(name):
    return os.path.join(shp_dir, name + '.shp')

# save removal function 
def safe_remove(path):
    if os.path.exists(path):
        os.remove(path)

# extract CRS from DEM and use it for reprojection of the pour points
with rasterio.open(tif('final_merged_output')) as src:
    src_crs = src.crs

**Cell 4:** DEM prepocessing pipeline 
Avoid running this multiple times, skip after you are happy with the result but are rerunning this script.

In [33]:
# Breach depressions (and fill depressions that are not breached)
wbt.breach_depressions_least_cost(
    dem=tif('final_merged_output'),
    output=tif('01_depression_breached'),
    dist=10000,
    fill = True,
    max_cost=None
    )


#Flow accumulation
wbt.d8_flow_accumulation(
    i=tif('01_depression_breached'),
    output=tif('02_flow_accumulation'),
    out_type='cells'
)


# Create D8 pointer raster
wbt.d8_pointer(
    dem=tif('01_depression_breached'),
    output=tif('03_d8_pointer'),
)

print(f'Created d8 pointer tif successfully! Saved it in {tif('05_d8_pointer')}')

.\whitebox_tools.exe --run="BreachDepressionsLeastCost" --wd="C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis" --dem='C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis\tif_edits\final_merged_output.tif' --output='C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis\tif_edits\01_depression_breached.tif' --dist='10000' --min_dist --fill -v --compress_rasters=False

*****************************************
* Welcome to BreachDepressionsLeastCost *
* Powered by WhiteboxTools              *
* www.whiteboxgeo.com                   *
*****************************************
Reading data...
Finding pits: 0%
Finding pits: 1%
Finding pits: 2%
Finding pits: 3%
Finding pits: 4%
Finding pits: 5%
Finding pits: 6%
Finding pits: 7%
Finding pits: 8%
Finding pits: 9%
Finding pits: 10%
Finding pits: 11%
Finding pits: 12%
Finding pits: 13%
Finding pits: 14%
Finding pits: 15%
Finding pits: 16%
Fi

_Optional_ : **Cell 5:** Generate interactive map to select pour points
After running this cell:
- click on the leaflet map below the code cell and add various amounts of pour points to the map (by clicking the marker on left hand side)
- click "Export" to download pour_points.geojson DO NOT CHANGE THE NAME (if you do you have to change the code in the next cell)

**If you don't do this import your own pour points shapefile (!) and change the name of the file to pour_points.shp. Save it in the filder shp_edits (as defined in Cell 3)**

In [34]:
# Use smoothed DEM to center the map
dem_path = tif('01_depression_breached')

# Reproject bounds to WGS84 (lat/lon)
with rasterio.open(dem_path) as src:
    wgs84_bounds = transform_bounds(src.crs, 'EPSG:4326', *src.bounds)
    minx, miny, maxx, maxy = wgs84_bounds
    center = [(miny + maxy) / 2, (minx + maxx) / 2]  # lat, lon


# Create interactive map
m = folium.Map(location=center, zoom_start=13, tiles="OpenTopoMap")

# Allow drawing multiple pour points
draw = plugins.Draw(
    export=True,
    filename='pour_points.geojson',
    draw_options={
        'polyline': False,
        'polygon': False,
        'circle': False,
        'rectangle': False,
        'circlemarker': False,
        'marker': True
    },
    edit_options={'edit': True}
)
draw.add_to(m)

# Show lat/lon on click
m.add_child(folium.LatLngPopup())

m

**Run only if you ran cell 5.**

**Cell 6:** Convert marked points on leaflet map to shapefile
Make sure you **saved the pour points in the workspace you defined in cell 3!**

In [35]:
# After downloading pour_points.geojson manually to the workspace:
geojson_path = os.path.join(workspace,  'pour_points.geojson')
shapefile_path = shp('pour_points')

# Load and convert to shapefile
gdf = gpd.read_file(geojson_path)
gdf.set_crs(epsg=4326, inplace=True)
gdf = gdf.to_crs(src_crs)
gdf.to_file(shapefile_path)

print(f"Saved pour points to: {shapefile_path}")

Saved pour points to: C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis\shp_edits\pour_points.shp


Run only if you ran cell 5 & 6. 

**Cell 7:** Snap pour points to flow accumulation grid and delineate watersheds:
Edit the snap_dist to an appropiate number. If you very sure about the location where you clicked then reduce the snap_dist, otherwise increase it. Snap created pour points to cell with largest accumulated flow. Units must be the same as the input raster (Meters for projected CRSs)

In [37]:
pourPoints = gpd.read_file(shp('pour_points'))
print(f'CRS of the pour points shapefile is: {pourPoints.crs}')

flowAcc = rasterio.open(tif('02_flow_accumulation'))
print(f'CRS of the Flow Accumulation raster is: {flowAcc.crs}')

#must be the same output EPSG

wbt.snap_pour_points(
    pour_pts=shp('pour_points'),
    flow_accum=tif('02_flow_accumulation'),
    output=shp('snapped_pour_points'),
    snap_dist=2000 # Unit is the unit of the DEM. Adjust snap distance as needed (in degrees for EPSG:4326 0.001 roughly 100m at equator)
)

print(f'Snapped pour points shapefile successfully! Saved them in {shp('snapped_pour_points')}')


CRS of the pour points shapefile is: EPSG:2193
CRS of the Flow Accumulation raster is: EPSG:2193
.\whitebox_tools.exe --run="SnapPourPoints" --wd="C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis" --pour_pts='C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis\shp_edits\pour_points.shp' --flow_accum='C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis\tif_edits\02_flow_accumulation.tif' --output='C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis\shp_edits\snapped_pour_points.shp' --snap_dist='2000' -v --compress_rasters=False

*****************************
* Welcome to SnapPourPoints *
* Powered by WhiteboxTools  *
* www.whiteboxgeo.com       *
*****************************
Reading data...
Progress: 0%
Progress: 100%
Saving data...
Output file written
Elapsed Time (excluding I/O): 0.116s
Snapped pour points shapefile successfully! Saved them i

**Cell 8:** Extract the watershed

In [ ]:
wbt.watershed(
    d8_pntr=os.path.abspath(tif('03_d8_pointer')),  
    pour_pts=os.path.abspath(shp('snapped_pour_points')),
    output=os.path.abspath(tif('06_watersheds'))
)

# Convert watershed raster to vector polygons
wbt.raster_to_vector_polygons(
    i = tif('06_watersheds'),
    output=shp('watershed_of_pour_points')
)

# Extract basins from the D8 pointer raster
# This will create a raster where each basin is assigned a unique value
# this works by asessing the d8 pointer raster and assigning unique values to each basin
# Note: This is different from the watershed extraction which creates a raster for each pour point
wbt.basins(
    d8_pntr=tif('03_d8_pointer'),
    output=tif('basins')
)
print(f'Created watersheds tif successfully! Saved it in {tif('06_watersheds')}')


.\whitebox_tools.exe --run="Watershed" --wd="C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis" --d8_pntr='C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis\tif_edits\03_d8_pointer.tif' --pour_pts='C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis\shp_edits\snapped_pour_points.shp' --output='C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis\tif_edits\06_watersheds.tif' -v --compress_rasters=False



****************************
* Welcome to Watershed     *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Locating pour points: 0%
Locating pour points: 100%
Initializing: 0%
Initializing: 1%
Initializing: 2%
Initializing: 3%
Initializing: 4%
Initializing: 5%
Initializing: 6%
Initializing: 7%
Initializing: 8%
Initializing: 9%
Initializing: 10%
Initializing: 11%
Initializing: 12%
Initializing: 13%
Initializing: 14%
Initializing: 15%
Initializing: 16%
Initializing: 17%
Initializing: 18%
Initializing: 19%
Initializing: 20%
Initializing: 21%
Initializing: 22%
Initializing: 23%
Initializing: 24%
Initializing: 25%
Initializing: 26%
Initializing: 27%
Initializing: 28%
Initializing: 29%
Initializing: 30%
Initializing: 31%
Initializing: 32%
Initializing: 33%
Initializing: 34%
Initializing: 35%
Initializing: 36%
Initializing: 37%
Initializing: 38%
Initializing: 39%
Initializing: 40%
Initializing: 41%
Initializing: 42%
Initializing: 43%
Initi

**Cell 9:** Extract stream network and main stem

In [46]:
# You may want to adjust the threshold value depending on your DEM and catchment size
threshold = 1000  # Minimum number of upstream cells to define a stream

wbt.extract_streams(
    flow_accum=tif('02_flow_accumulation'),
    output=tif('streams_raster'),
    threshold=threshold
)

wbt.raster_streams_to_vector(
    streams = tif('streams_raster'), 
    d8_pntr = tif('03_d8_pointer'), 
    output = shp('streams_vector'), 
    esri_pntr=False
)
print(f'Stream raster extracted and saved to: {shp('streams_vector')}')

streams_vector = gpd.read_file(shp('streams_vector'))
streams_vector = streams_vector.set_crs(src_crs)  # Set CRS to match the DEM
streams_vector.to_file(shp('streams_vector'), driver='ESRI Shapefile')


wbt.find_main_stem(
    d8_pntr = tif('03_d8_pointer'), 
    streams = tif('streams_raster'), 
    output = tif('main_stem_raster'), 
    esri_pntr=False, 
    zero_background=False
)

wbt.raster_streams_to_vector(
    streams = tif('main_stem_raster'), 
    d8_pntr = tif('03_d8_pointer'), 
    output = shp('main_stem_vector'), 
    esri_pntr=False
)
print(f'Stream raster extracted and saved to: {shp('main_stem_vector')}')

streams_vector = gpd.read_file(shp('main_stem_vector'))
streams_vector = streams_vector.set_crs(src_crs)  # Set CRS to match the DEM
streams_vector.to_file(shp('main_stem_vector'), driver='ESRI Shapefile')


.\whitebox_tools.exe --run="ExtractStreams" --wd="C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis" --flow_accum='C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis\tif_edits\02_flow_accumulation.tif' --output='C:\Users\Raphael Menke\Documents\programmieren\python\pythonPractise\catchmentAnalysis\tif_edits\streams_raster.tif' --threshold='1000' -v --compress_rasters=False

*****************************
* Welcome to ExtractStreams *
* Powered by WhiteboxTools  *
* www.whiteboxgeo.com       *
*****************************


Reading data...
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress: 36%
Progress: 37%
Progress: 38%
Progress: 39%
Progress: 40%
Progress: 41%
Progress: 42%
Progress: 43%
Progress: 44%
Progress: 45%
Progress: 46%
Progress: 47%
Progress: 48%
Progress: 49%
Progress: 50%
Progress: 51%
Progress: 52%
Progress: 53%
Progress: 54%
Progress: 55%
Progress: 56%
Progress: 57%
Progress: 58%
Progress: 59%
Progress: 60%
Progress: 61%
Progress: 62%
Progress: 63%
Progress: 64%
Progress: 65%
Progress: 66%
Progress: 67%
Progress: 68%
Progress: 69%
Progress: 70%
